In [1]:
import os
import streamlit as st
from dotenv import load_dotenv
from ticket_master_utils import search_events, parse_event
from spotify_utils import authenticate_to_spotify, get_current_user_top_artists
from streamlit_card import card
from collections import defaultdict



In [2]:

def group_events_by_artist(events):
    artist_events = defaultdict(list)
    for event in events:
        artist_name = event['band_name'].lower()
        artist_events[artist_name].append(event)

    artist_events_dict = dict(artist_events)

    return artist_events_dict

def beutify_event(event):
    with st.expander("Event Details", expanded=False):
        st.title("Concert Details")

        st.header(event["band_name"])
        st.subheader(event["event_name"])
        st.write(f"Date and Time: {event['start_datetime']} ({event['timezone']})")
        st.write(f"Venue: {event['venue_name']}")
        st.write(f"Address: {event['venue_address']}")
        st.write(f"Event URL: [{event['event_name']}]({event['event_url']})")
    


In [7]:

load_dotenv()
ticketmaster_api_key = os.getenv('TICKETMASTER_API_KEY')
ticketmaster_api_url = os.getenv('TICKETMASTER_API_URL')

client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIFY_REDIRECT_URI')
scope = os.getenv('SPOTIFY_APP_SCOPE')


# st.title('Content Finder')

sp = None

# st.markdown(body='Please connect to your Spotify account')
sp_connect_button = st.button(label='Connect', key='connect_spotify_button')
events_to_get = st.slider(min_value=1, max_value=10, value=1, step=1, label='events_to_get')

all_artists = []
# if sp_connect_button:
try:
    sp = authenticate_to_spotify(client_id, client_secret, redirect_uri, scope)

except Exception as e:
    # st.error(e)
    print(e)

finally:
    top_artists = get_current_user_top_artists(sp, limit=500)

    for artist in top_artists:
        artist_events = []
        events = search_events(artist, ticketmaster_api_key, ticketmaster_api_url, events_to_get)
    
        if events:
            for event in events:
                if event:
                        event = parse_event(band_name=artist, event=event)
                        artist_events.append(event)

            all_artists.append(artist_events)

            for artist_ev in all_artists:
                band = artist_ev[0]['band_name']
                st.text(band)
                x = 0
                for event in artist_ev:
                    # with st.expander(f"{band}", expanded=False):
                        text = [f'Band: {event["band_name"]}', f'Event: {event["event_name"]}', f'Date and Time: {event["date_and_time"]}', f'Venue: {event["venue_name"]}', f'Address: {event["venue_address"]}']

                        # card(title=event["event_name"],
                        #         text= text,
                        #         url=f"{event['event_url']}",
                        #         styles= {
                        #             "card": {
                        #                 "width": "700px",
                        #                 "height": "450px"
                        #                 }
                        #             },
                        #         image=event["image"], 
                        #         key=(f'{event["band_name"]}+{x}'),
                        #         )
                        # x += 1


In [8]:
text

['Band: Rammstein',
 'Event: Rammstein + Ghost Tribute Double Header - Rammlied & Popestars',
 'Date and Time: 2023-10-13 20:00:00 (Europe/London)',
 'Venue: Nightrain',
 'Address: 7-19 Queensgate']

In [9]:
artist_ev

[{'band_name': 'Rammstein',
  'event_name': 'Rammstein + Ghost Tribute Double Header - Rammlied & Popestars',
  'date_and_time': '2023-10-13 20:00:00 (Europe/London)',
  'start_datetime': 'Friday, October 13, 2023 at 08:00 PM',
  'event_url': 'https://www.ticketweb.uk/event/rammstein-ghost-tribute-double-nightrain-tickets/12542155?REFERRAL_ID=tmfeed',
  'timezone': 'Europe/London',
  'locale': 'en-us',
  'venue_name': 'Nightrain',
  'venue_address': '7-19 Queensgate',
  'image': 'https://s1.ticketm.net/dam/c/e7c/86478562-7218-4e8f-b9d4-8080fe168e7c_105581_TABLET_LANDSCAPE_LARGE_16_9.jpg'}]

In [10]:
all_artists

[[{'band_name': 'The Devil Wears Prada',
   'event_name': 'The Devil Wears Prada & Fit For A King: METALCORE DROPOUTS',
   'date_and_time': '2023-10-07 19:00:00 (America/New_York)',
   'start_datetime': 'Saturday, October 07, 2023 at 07:00 PM',
   'event_url': 'https://concerts.livenation.com/the-devil-wears-prada-fit-for-charlotte-north-carolina-10-07-2023/event/2D005EA9DAE388C5',
   'timezone': 'America/New_York',
   'locale': 'en-us',
   'venue_name': 'The Fillmore Charlotte',
   'venue_address': '820 Hamilton Street',
   'image': 'https://s1.ticketm.net/dam/a/b3a/f1339630-a8e8-401a-9d75-c3ad2f434b3a_1684791_TABLET_LANDSCAPE_3_2.jpg'}],
 [{'band_name': 'Architects',
   'event_name': 'Architects',
   'date_and_time': '2024-01-25 18:30:00 (Europe/Brussels)',
   'start_datetime': 'Thursday, January 25, 2024 at 06:30 PM',
   'event_url': 'https://www.ticketmaster.be/event/architects-tickets/57853',
   'timezone': 'Europe/Brussels',
   'locale': 'en-us',
   'venue_name': 'Lotto Arena Ant